In [ ]:
%pip install --upgrade git+https://github.com/huggingface/transformers.git 
!pip install huggingface_hub
!sudo apt-get update
!sudo apt-get install -y poppler-utils


In [6]:
%pip install byaldi  accelerate  qwen_vl_utils pdf2image

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 67.3 MB/s eta 0:00:00:00:01
Note: you may need to restart the kernel to use updated packages.


In [7]:
from huggingface_hub import notebook_login
notebook_login()

In [1]:
from byaldi import RAGMultiModalModel
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch
from pdf2image import convert_from_path

RAG = RAGMultiModalModel.from_pretrained("vidore/colqwen2-v1.0")


Verbosity is set to 1 (active). Pass verbose=0 to make quieter.


model-00001-of-00002.safetensors:   2%|2         | 105M/4.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.85G [00:00<?, ?B/s]

`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/392 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [2]:
RAG.index(
    input_path="/home/Redacted.pdf",
    index_name="index",
    store_collection_with_index=False,
    overwrite=True
)

images = convert_from_path("/home/Redacted.pdf")

Added page 1 of document 0 to index.
Added page 2 of document 0 to index.
Added page 3 of document 0 to index.
Added page 4 of document 0 to index.
Added page 5 of document 0 to index.
Added page 6 of document 0 to index.
Added page 7 of document 0 to index.
Added page 8 of document 0 to index.
Added page 9 of document 0 to index.
Added page 10 of document 0 to index.
Added page 11 of document 0 to index.
Added page 12 of document 0 to index.
Added page 13 of document 0 to index.
Added page 14 of document 0 to index.
Added page 15 of document 0 to index.
Added page 16 of document 0 to index.
Added page 17 of document 0 to index.
Added page 18 of document 0 to index.
Added page 19 of document 0 to index.
Added page 20 of document 0 to index.
Added page 21 of document 0 to index.
Added page 22 of document 0 to index.
Added page 23 of document 0 to index.
Added page 24 of document 0 to index.
Added page 25 of document 0 to index.
Added page 26 of document 0 to index.
Added page 27 of docu

In [3]:
search_query = input("Enter the question")

results = RAG.search(search_query, k=1)
print(results)

[{'doc_id': 0, 'page_num': 69, 'score': 17.5, 'metadata': {}, 'base64': None}]


In [4]:

image_indexes = [pages['page_num']-1 for pages in results]

LLAM3.2 90B

In [9]:
from transformers import MllamaForConditionalGeneration, AutoProcessor, MllamaProcessor

model_id = "meta-llama/Llama-3.2-90B-Vision-Instruct"

llama_model = MllamaForConditionalGeneration.from_pretrained(
    model_id,
    device_map = "auto",
    # token = ""
)
processor = AutoProcessor.from_pretrained(model_id, 
# token = ""
)

config.json:   0%|          | 0.00/5.19k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/147k [00:00<?, ?B/s]

model-00001-of-00037.safetensors:   0%|          | 0.00/4.60G [00:00<?, ?B/s]

model-00002-of-00037.safetensors:   0%|          | 0.00/4.66G [00:00<?, ?B/s]

model-00003-of-00037.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
def get_images(image_indexes):
    return[images[i] for i in image_indexes]

image_arr = get_images(image_indexes)
    
messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": """Answer the following question from the given image 
            question : {search_query}
        
        """}
    ]}
]
input_text = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(
    # images[image_indexes[0]],
    image_arr,
    input_text,
    add_special_tokens=False,
    return_tensors="pt",
).to(model.device)

output = model.generate(**inputs, max_new_tokens=30)
print(processor.decode(output[0]))